This code is taking the image from the dji tello drone, and then passing it through depth anything model and then saving it to a folder

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from djitellopy import Tello
import time
import cv2
import numpy as np
import os
import torch
import torch.nn.functional as F
from torchvision.transforms import Compose
from tqdm import tqdm
from depth_anything.dpt import DepthAnything
from depth_anything.util.transform import Resize, NormalizeImage, PrepareForNet



2024-04-03 21:07:25.567773: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 21:07:25.596715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 21:07:25.596749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 21:07:25.598113: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 21:07:25.603092: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tello = Tello()
tello.connect()
print(tello.query_battery())

[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'battery?'
[INFO] tello.py - 462 - Response battery?: '56'


56


In [3]:
tello.streamon()
frame_read = tello.get_frame_read()
# tello.takeoff()
# tello.move_up(100)
# cv2.imwrite("picture.png", frame_read.frame)



[INFO] tello.py - 438 - Send command: 'streamon'
[INFO] tello.py - 462 - Response streamon: 'ok'
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-exist

In [4]:
encoder = 'vits'
video_path = 1
margin_width = 50
caption_height = 60

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
depth_anything = DepthAnything.from_pretrained('LiheYoung/depth_anything_{}14'.format(encoder)).to(DEVICE)
depth_anything.eval()
transform = Compose([
    Resize(
        width=518,
        height=518,
        resize_target=False,
        keep_aspect_ratio=True,
        ensure_multiple_of=14,
        resize_method='lower_bound',
        image_interpolation_method=cv2.INTER_CUBIC,
    ),
    NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    PrepareForNet(),
])


xFormers not available
xFormers not available


Function for testing the Aux function

In [5]:
# img = cv2.imread("./testing_data/D2-L1-0.png")
# img = preprocess_frame(img)
# action, action_list = policy_REINFORCE(img, agent)                            #action->np.array,(1,1)
# action_word = translate_action([action], 25)


Actual Inference Happening

In [ ]:
start = time.time()
i=0

while time.time()-start < 30:
    frame = tello.get_frame_read().frame
    frame = cv2.resize(frame, (640, 480))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) / 255.0
    h, w = frame.shape[:2]
    image = transform({'image': frame})['image']
    image = torch.from_numpy(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        depth = depth_anything(image)
    depth = F.interpolate(depth[None], (h, w), mode='bilinear', align_corners=False)[0, 0]
    depth = (depth - depth.min()) / (depth.max() - depth.min()) * 255.0
    
    depth = depth.cpu().numpy().astype(np.uint8)
    depth_color = cv2.applyColorMap(depth, cv2.COLORMAP_INFERNO)
    # Our operations on the frame come here
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Display the resulting frame
    path = "depth_output/frame"+str(i)+".jpg"
    cv2.imwrite(path,depth_color)
    i+=1
    
    
    
   

left block unavailable for requested intra mode
error while decoding MB 0 16, bytestream 8818
left block unavailable for requested intra mode
error while decoding MB 0 28, bytestream 5024
error while decoding MB 2 44, bytestream -8
left block unavailable for requested intra mode
error while decoding MB 0 13, bytestream 10258


[INFO] tello.py - 438 - Send command: 'land'


[INFO] tello.py - 462 - Response land: 'ok'
